# Agents - Make OpenAI Do Things For you

*[Source](https://langchain.readthedocs.io/en/latest/modules/agents/getting_started.html)*
* **Agent** - Agents use an LLM to determine which actions to take and in what order. An action can either be using a tool and observing its output, or returning to the user.

Parameters when creating an agent:
* **Tool:** A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains. The interface for a tool is currently a function that is expected to have a string as an input, with a string as an output.
* **LLM:** The language model powering the agent.
* **Agent:** The agent to use. This should be a string that references a support agent class. Because this notebook focuses on the simplest, highest level API, this only covers using the standard supported agents. If you want to implement a custom agent, see the documentation for custom agents (coming soon).

In [13]:
!pip3 install wikipedia langchain openai google-search-results
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms.openai import AzureOpenAI

DEPRECATION: pytorch-lightning 1.6.4 has a non-standard dependency specifier torch>=1.8.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pytorch-lightning or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


### List of Tools
1. **python_repl -** A Python shell. Use this to execute python commands. Input should be a valid python command. If you expect output it should be printed out.
2. **serpapi [Complete] -** A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
3. **wolfram-alpha [Complete] -** A wolfram alpha search engine. Useful for when you need to answer questions about Math, Science, Technology, Culture, Society and Everyday Life. Input should be a search query.
4. **requests -** A portal to the internet. Use this when you need to get specific content from a site. Input should be a specific url, and the output will be all the text on that page.
5. **terminal -** Executes commands in a terminal. Input should be valid commands, and the output will be any output from running that command.
6. **pal-math -** A language model that is excellent at solving complex word math problems. Input should be a fully worded hard word math problem.
7. **pal-colored-objects -** A language model that is wonderful at reasoning about position and the color attributes of objects. Input should be a fully worded hard reasoning problem. Make sure to include all information about the objects AND the final question you want to answer.
8. **llm-math -** Useful for when you need to answer questions about math.
9. **open-meteo-api -** Useful for when you want to get weather information from the OpenMeteo API. The input should be a question in natural language that this API can answer.
10. **news-api -** Use this when you want to get information about the top headlines of current news stories. The input should be a question in natural language that this API can answer.
11. **tmdb-api -** Useful for when you want to get information from The Movie Database. The input should be a question in natural language that this API can answer.
12. **google-search -** A wrapper around Google Search. Useful for when you need to answer questions about current events. Input should be a search query.


### 2. Imports and env setup.

In [14]:
from langchain.agents import load_tools
import os
import tiktoken
import openai
import numpy as np
from dotenv import load_dotenv

# Load environment variables
load_dotenv('../.env', override=True)

# Option 2 - Using Access Key
openai.api_type = "azure"
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get('OPENAI_API_VERSION', "2022-12-01")

# Define embedding model and encoding
EMBEDDING_MODEL = os.environ.get('OPENAI_EMBEDDING_MODE', 'text-embedding-ada-002')
OPENAI_COMPLETION_MODEL = os.environ.get('OPENAI_COMPLETION_MODEL', 'gpt-35-turbo')

#This are the OPENAI variables we shall use:
OPENAI_VARS = [(key, value) for key, value in os.environ.items() if 'openai' in key.lower()]
print(OPENAI_VARS)


[('OPENAI_API_KEY', '8158d342aa794004ae7a016cb6e8033a'), ('OPENAI_COMPLETION_MODEL', 'gpt-35-turbo'), ('OPENAI_EMBEDDINGS_API_VERSION', '2023-07-01-preview'), ('OPENAI_API_API_TYPE', 'azure'), ('OPENAI_EMBEDDING_MODEL', 'text-embedding-ada-002'), ('OPENAI_API_VERSION', '2022-12-01'), ('OPENAI_API_BASE', 'https://handsonoctober.openai.azure.com'), ('OPENAI_API_REGION', 'eastus')]


In [15]:
#Lets init an Azure Open AI instance
llm = AzureOpenAI(
    deployment_name=OPENAI_COMPLETION_MODEL, temperature=0, openai_api_version=openai.api_version)

#llm = OpenAI(temperature=0)

In [16]:

tool_names = ['python_repl', "serpapi", 'wikipedia']
tools = load_tools(tool_names)

In [17]:
# What is python_repl
tools[0].name, tools[0].description

('Python_REPL',
 'A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.')

In [18]:
# What is serpapi?
tools[1].name, tools[1].description

('Search',
 'A search engine. Useful for when you need to answer questions about current events. Input should be a search query.')

In [19]:
#Create an agent with the defined tools.
agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

In [20]:
agent.run("Create python to print this string: 1+2=3")



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


 I can just print the string
Action: Python_REPL
Action Input: print("1+2=3")
Observation: 1+2=3

Thought: I have the answer
Final Answer: 1+2=3

> Finished chain.


'1+2=3'

In [21]:
# Input should be a search query.
agent.run("who is the ceo of pipe?")



> Entering new AgentExecutor chain...
 I don't know who the CEO of Pipe is, so I should use Wikipedia to find out.
Action: Wikipedia
Action Input: "Pipe Technologies"